In [14]:
import os
import pandas as pd
import json
from utils import decoder_function
from dataloader import CustomTextDataset

## About
convert the text into integers and character mapping

### TODO
- Join all the train, test and val so all the character in the dataset is used


In [6]:
## 
DATASET_PATH = os.path.join("..","dataset","dakshina_dataset_v1.0","ta","lexicons")
TRAIN = "ta.translit.sampled.train.tsv"
VAL = "ta.translit.sampled.dev.tsv"
TEST =   "ta.translit.sampled.test.tsv"

In [3]:
all_lines = []
for datasplit in [TRAIN, VAL,TEST]:
    with open(os.path.join(DATASET_PATH, datasplit), "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
        lines = [i for i in lines if len(i)>0]
        print(datasplit, len(lines))
        all_lines.extend(lines)

ta.translit.sampled.train.tsv 68218
ta.translit.sampled.dev.tsv 6827
ta.translit.sampled.test.tsv 6864


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
num_samples = 10
for line in all_lines:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


## Character to index this is for the embedding layer
english_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
tamil_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

with open(os.path.join(DATASET_PATH,"english_token_index.json"), "w", encoding="utf-8") as file:
	json.dump(english_token_index, file, indent=4)
with open(os.path.join(DATASET_PATH,"tamil_token_index.json"), "w", encoding="utf-8") as file:
	json.dump(tamil_token_index, file, indent=4)



Number of samples: 81909
Number of unique input tokens: 26
Number of unique output tokens: 48
Max sequence length for inputs: 30
Max sequence length for outputs: 28


In [ ]:
# Number of samples: 81909
# Number of unique input tokens: 46
# Number of unique output tokens: 28
# Max sequence length for inputs: 26
# Max sequence length for outputs: 32

In [7]:
# How to load the dict
# with open(os.path.join(DATASET_PATH,"input_token_index.json"),'r', encoding="utf-8") as f:
# 	data = json.load(f)
# print(data)

In [5]:
for datasplit in [TRAIN, VAL,TEST]:
    tamil_char_idx_list = []
    eng_char_idx_list = []
    with open(os.path.join(DATASET_PATH, datasplit), "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
        for line in all_lines:
            target_text,input_text, _ = line.split("\t")
            target_text = "\t" + target_text + "\n"

            tamil_char_idx_list.append(",".join(str(tamil_token_index[i]) for i in target_text))
            eng_char_idx_list.append(",".join(str(english_token_index[i]) for i in input_text))
    char_idx_df = pd.DataFrame.from_dict({"Tamil":tamil_char_idx_list,"English":eng_char_idx_list})
    char_idx_df.to_csv(os.path.join(DATASET_PATH, datasplit.replace(".tsv",".idx.csv")),index=False)


## Save the mapping dict


In [7]:
char_idx_df = pd.read_csv(os.path.join(DATASET_PATH, "ta.translit.sampled.train.idx.csv"))


with open(os.path.join(DATASET_PATH,"tamil_token_index.json"),'r', encoding="utf-8") as f:
	tamil_idx = json.load(f)
tamil_idx_to_char = {j:i for i,j in tamil_idx.items()}

with open(os.path.join(DATASET_PATH,"english_token_index.json"),'r', encoding="utf-8") as f:
	english_idx = json.load(f)
english_idx_to_char = {j:i for i,j in english_idx.items()}

In [15]:


char_idx_df["Tamil_string"] = char_idx_df["Tamil"].apply(lambda x: decoder_function(character_idx_seq=x, idx_to_char_dict=tamil_idx_to_char))
char_idx_df["English_string"] = char_idx_df["English"].apply(lambda x: decoder_function(character_idx_seq=x, idx_to_char_dict=english_idx_to_char))

char_idx_df.head()

,Tamil,English,Tamil_string,English_string
0,"0,2,24,37,26,19,47,1","5,8,0,19",\tஃபியட்\n,fiat
1,"0,2,24,37,26,19,47,1","15,7,8,24,0,19",\tஃபியட்\n,phiyat
2,"0,2,24,37,26,19,47,1","15,8,24,0,19",\tஃபியட்\n,piyat
3,"0,2,24,37,27,36,23,47,34,47,1","5,8,17,0,0,13,18",\tஃபிரான்ஸ்\n,firaans
4,"0,2,24,37,27,36,23,47,34,47,1","5,17,0,13,2,4",\tஃபிரான்ஸ்\n,france


In [9]:
text_dataset = CustomTextDataset(dataset_df = char_idx_df)

In [20]:
x,y = text_dataset.__getitem__(867)
print(decoder_function(character_idx_seq=','.join([str(i) for i in x]), idx_to_char_dict=english_idx_to_char))
print(decoder_function(character_idx_seq=','.join([str(i) for i in y]), idx_to_char_dict=tamil_idx_to_char))

adaiyaalam
	அடையாளம்



In [12]:
y

[0, 2, 24, 37, 27, 36, 23, 47, 34, 47, 1]